In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from tensorflow.python.client import device_lib
import pandas as pd
import re
import numpy as np

import requests
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from timeit import default_timer as timer
import math
import random
from collections import defaultdict
from keras.models import model_from_json
import h5py
import numpy as np
import sentencepiece as spm
from sklearn.model_selection import train_test_split
import sentencepiece as sp
random.seed(11)

Using TensorFlow backend.


In [2]:
print(tf.__version__)

1.15.0


In [3]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
sp = spm.SentencePieceProcessor(model_file='book.model')

In [5]:
with open("testbook.txt") as handle:
    sentences =  [l.strip() for l in handle.readlines()]
    
v = 3000

In [6]:
print(len(sentences))

2063


In [7]:

x = []

for a in sentences:
    if len(a.split()) > 2 and len(a.split()) < 70:
        x.append(a)

In [8]:
X = []
X1 = []
y1 = []
for sentence in x:
    X.append(sp.encode(sentence))
    
for sentence in X:
    y1.append(sentence[-1])
    X1.append(sentence[0:-1])
    
print(y1[0])
print(X1[0])
    
y = to_categorical(y1, num_classes=v)

399
[44, 80, 77, 79, 394, 1724, 9, 61, 76, 961, 3, 48, 1391, 820, 605, 916, 390, 5, 935, 46, 34, 4, 559, 9, 83, 418, 83, 825, 45, 59, 873, 6, 25, 1278, 111, 59, 126, 1894, 914, 93, 1314]


In [9]:
print(sp.decode(X1[0]))
print(sp.decode(y1[0]))   

The Project Gutenberg EBook of A Christmas Carol, by Charles Dickens This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatso
ever


In [10]:
max1 = 0
for i in x:
    if len(i) > max1:
        max1 = len(i)
print(max1)

405


In [14]:
X2 = []
for XD in X1:
    X2.append(pad_sequences([XD], maxlen=max1, padding='pre')[0])
    
X2 = np.asarray(X2)
X = X2
X



array([[   0,    0,    0, ...,  914,   93, 1314],
       [   0,    0,    0, ...,  354,    8,  553],
       [   0,    0,    0, ...,    3,  152, 2604],
       ...,
       [   0,    0,    0, ..., 2046,   46,  968],
       [   0,    0,    0, ...,   14,   13,  400],
       [   0,    0,    0, ...,  318,  107,  577]])

In [20]:
c = list(zip(X, y))

random.shuffle(c)

X, y = zip(*c)

X = np.asarray(X)
y = np.asarray(y)
X

array([[   0,    0,    0, ..., 2862, 1566,   28],
       [   0,    0,    0, ...,   51,  137,   12],
       [   0,    0,    0, ...,   12,  114, 1169],
       ...,
       [   0,    0,    0, ...,   24,  265,   65],
       [   0,    0,    0, ...,   63,  219,   42],
       [   0,    0,    0, ...,    0,  186,  140]])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [22]:
a = []
u = 94

for i in X_train[u][np.nonzero(X_train[u])[0][0]:]:
    a.append(int(i))
    
    
print(sp.decode(a))
print(sp.decode(int(np.nonzero(y_train[u])[0][0])))   

'Poor Robin Crusoe, where have you been, Robin Crusoe
'


# Model

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras import backend as K
import scipy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from keras.callbacks import EarlyStopping
import time

In [55]:
def create_model(optimizer='adam'):
    K.clear_session()
    model = Sequential()
    
    model.add(Embedding(v, 200, input_length = max1, mask_zero=True))

    
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))

    # output layer
    model.add(Dense(v, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Create param_grid which is a dictionary of parameters to test
# only using single parameters right now to speed up the proces
model = KerasClassifier(build_fn=create_model, verbose=0)
# Add things to these lists to add parameters to be tested
epochs = [10]
batches = [128]
optimizers = ['adam']

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [56]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [57]:
start = time.time()
grid_result = grid.fit(X_test, y_test, callbacks=[early_stopping], validation_data=(X_test, y_test))
end = time.time()
print(end - start)

309.93455719947815


In [58]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.195815 using {'batch_size': 128, 'epochs': 10, 'optimizer': 'adam'}
0.195815 (0.069754) with: {'batch_size': 128, 'epochs': 10, 'optimizer': 'adam'}


In [43]:
def perplexity(Xp, yp, model):
    chance = 0
    current_index = 0
    batch_size = 205

    for _ in range(50):
        distributionList = model.predict(Xp[current_index:current_index + batch_size], batch_size)
    
        tempcount = current_index
        for distribution in distributionList:
            chance += math.log2(distribution[np.nonzero(yp[tempcount])[0][0]])
            tempcount += 1
            
        current_index = current_index + batch_size 
    return 2**((-1/len(Xp))* chance)

In [ ]:
def train_model(model, epochs, batch_size):

 # kernel_regularizer=tf.keras.regularizers.l1(0.01)
    for i in range(30):
        model.fit(X_train, y_train, epochs=1, verbose=1, batch_size=128)
        print("Loss and accuracy on valid")
        print(model.evaluate(x=X_test, y=y_test, batch_size=128, verbose=0))
        print("Perplexity:")
        print(perplexity(X_test, y_test, model))
        model.save_weights(str(i) + ".h5")
        print("Saved model to disk")
        print("")
    return model

model = train_model(model, 50, 256)

In [ ]:
model.load_weights("5 model.h5")

Make predictions using this function

In [35]:
def generate_words(model, text):
    # create word embeddings
    words_embedding = sp.encode(sentence)

    padded_words = pad_sequences([words_embedding], maxlen=max1, padding='pre')
    # predict next word
    prediction = np.argmax(model.predict(padded_words), axis=-1)
    print(sp.decode(int(prediction[0])))
    print(sorted(model.predict(padded_words)[0], reverse=True)[0:5])
        

In [52]:
sentence = sp.decode([1054,    5,
       1892,  637,    4,    3,    6,   10,   17,    4,   74,   18,    3,
        621,   10,   11,  362,   23, 1894,   39,  300, 1909, 4165,  827,
          3,    6,  172,   70,   74,   18, 3670,    8,    9,    3,  132,
       1189,   24])
print(sentence)
generate_words(model, sentence)

From the collection centers , toys will be taken to a warehouse at 198 Second street , where they will be repaired and made ready for
it
[0.077863425, 0.072056495, 0.056057617, 0.02283041, 0.015822673]


In [28]:
with open('brownsent.txt', 'r') as file:
    data = file.read().replace('\n', '')

data = data.split()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [29]:
def baseline_perplex(data):
    distribution = defaultdict(int)
    for word in data:
        distribution[word] += 1
        
    probdistribution = []   
    sumD = sum(distribution.values())
    for num in list(distribution.values()):
        probdistribution.append(num/sumD)
        
    count = 0
    for key in distribution.keys():
        distribution[key] = probdistribution[count]
        count += 1

    chance = 0
    for i in range(len(y_train)):
        chance += math.log2(distribution.get(y_train[i], (1/sumD)))
    
    return 2**((-1/len(y_train))* chance)


baseline_perplex(X_train)

1582.812467959088

In [19]:
start = time.time()
print(perplexity(X_test, y_test, model))
end = time.time()
print(end - start)

226.9762551690742
74.30903339385986


In [26]:
#1 Final perplexity 250
#2 Final perplexity 542
#3 Final perplexity 1178
# Running model too long makes it overfit, as expected.

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
# This function is obsolete (much slower), keeping it here for archive purposes.

def perplexity1(Xp, yp):
    chance = 0
    for i in range(len(Xp)):   
        wordprob = model.predict([[Xp[i]]])[0][np.nonzero(yp[i])[0][0]]
        if wordprob != 0:
            chance += math.log2(wordprob)
        print(wordprob)
    return 2**((-1/len(Xp))* chance)

perplexity1(X_test[0:12], y_test)